In [1]:
import pandas as pd
#import sweetviz as sv
import os
import plotly.express as px
import seaborn as sns
import matplotlib as mlp
import psycopg2
import plotly.offline as pyo
import random
from dash import Dash, html, dcc, Input, Output
import dash_daq as daq
from flask import Flask
from gevent.pywsgi import WSGIServer
import numpy as np
import os, signal

In [2]:
def connection(sql):
    con = psycopg2.connect(host='35.247.202.234', database='dw',
    user='consulta', password='teste123')
    cur = con.cursor()    
    cur.execute(sql)
    data = cur.fetchall()
    cols = []
    for elt in cur.description:
        cols.append(elt[0])

    df = pd.DataFrame(data=data,columns=cols)
    con.close();
    return df

In [3]:
query = """select * from public.fatores_est fe
inner join escolas.aluno a 
on fe.id_aluno = a.id_aluno 
inner join escolas.turma t 
on a.id_turma = t.id_turma 
inner join escolas.escola e 
on t.id_escola = e.cod_escola"""
df = connection(query)


In [4]:
df = df.loc[:,~df.columns.duplicated()].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 60 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   id                          62 non-null     int64              
 1   id_aluno                    62 non-null     int64              
 2   E_ESC1V                     62 non-null     float64            
 3   E_ESC2V                     62 non-null     float64            
 4   E_PROF1V                    62 non-null     float64            
 5   E_PROF2V                    62 non-null     float64            
 6   E_FAM1V                     62 non-null     float64            
 7   E_FAM2V                     62 non-null     float64            
 8   E_COM1V                     62 non-null     float64            
 9   E_COM2V                     62 non-null     float64            
 10  E_COM3V                     62 non-null     float64            


In [5]:
df.columns

Index(['id', 'id_aluno', 'E_ESC1V', 'E_ESC2V', 'E_PROF1V', 'E_PROF2V',
       'E_FAM1V', 'E_FAM2V', 'E_COM1V', 'E_COM2V', 'E_COM3V', 'E_EST1V',
       'E_EST2V', 'E_EST3V', 'E_ESC1C', 'E_ESC2C', 'E_PROF1C', 'E_PROF2C',
       'E_FAM1C', 'E_FAM2C', 'E_COM1C', 'E_COM2C', 'E_COM3C', 'E_EST1C',
       'E_EST2C', 'E_EST3C', 'created_date', 'updated_date', 'id_turma',
       'cod_matricula', 'nome_turma', 'nome_aluno', 'tp_sexo', 'id_escola',
       'modo_ensino', 'etapa_ensino', 'ano_turma', 'data_criacao',
       'resticao_de_atendimento', 'escola', 'cod_escola', 'uf', 'municipio',
       'localizacao', 'localidade_diferenciada', 'categoria_administrativa',
       'endereco', 'telefone', 'dependencia_administrativa',
       'categoria_escola', 'conveniada_poder_publico',
       'regulamentacao_conselho', 'porte', 'etapas_e_modalidade_ensino',
       'outras_ofertas', 'cod_regiao', 'micro', 'meso', 'regiao',
       'cod_estado'],
      dtype='object')

In [6]:
cross_tab_prop = df[['id_aluno','nome_aluno','id_turma', 'escola', 'uf', 'municipio','cod_escola','nome_turma','E_ESC1V', 'E_ESC2V', 'E_PROF1V', 'E_PROF2V',
       'E_FAM1V', 'E_FAM2V', 'E_COM1V', 'E_COM2V', 'E_COM3V', 'E_EST1V',
       'E_EST2V', 'E_EST3V', 'E_ESC1C', 'E_ESC2C', 'E_PROF1C', 'E_PROF2C',
       'E_FAM1C', 'E_FAM2C', 'E_COM1C', 'E_COM2C', 'E_COM3C', 'E_EST1C',
       'E_EST2C', 'E_EST3C']]

cross_tab_prop.set_index('id_aluno')
cross_tab_prop

,id_aluno,nome_aluno,id_turma,escola,uf,municipio,cod_escola,nome_turma,E_ESC1V,E_ESC2V,...,E_PROF1C,E_PROF2C,E_FAM1C,E_FAM2C,E_COM1C,E_COM2C,E_COM3C,E_EST1C,E_EST2C,E_EST3C
0,14799,ADRYAN MAGNO DA SILVA NEVES,695,EEEFM DR OSWALDO PIANNA,RO,Porto Velho,11000937,None,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
1,18949,WEVERTON DE SOUZA DA SILVA,1059,E M E I F SANTA TEREZINHA,PA,Cametá,15547574,None,3.333333,3.0,...,3,2,3,2,1,1,3,2,2,1
2,19175,Monique Gomes de Souza,883,E M E F PROFESSORA DALILA LEAO,PA,Cametá,15070387,None,1.000000,3.0,...,1,1,1,1,1,1,1,2,1,1
3,19174,Elielson Rodrigues Viana,892,E M E F PROFESSORA DALILA LEAO,PA,Cametá,15070387,None,1.000000,4.0,...,1,1,1,2,1,1,3,2,1,3
4,19152,Jeferson Oliveira dos Santos,1064,E M E F PROFESSORA DALILA LEAO,PA,Cametá,15070387,None,3.000000,1.0,...,1,1,1,2,1,1,2,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,18920,ARLAN CARDOSO RODRIGUES,1059,E M E I F SANTA TEREZINHA,PA,Cametá,15547574,None,2.000000,1.0,...,1,1,3,1,1,3,1,2,1,1
58,18918,ALINE DA CRUZ MONTEIRO,1059,E M E I F SANTA TEREZINHA,PA,Cametá,15547574,None,1.333333,4.0,...,2,3,2,2,1,3,3,3,3,3
59,18917,ALEXANDRE LEÃO CANTÃO,1059,E M E I F SANTA TEREZINHA,PA,Cametá,15547574,None,5.500000,4.5,...,2,2,2,2,1,3,2,1,3,3
60,18916,ALESSANDRO PORTILHO DE MORAES,1059,E M E I F SANTA TEREZINHA,PA,Cametá,15547574,None,1.666667,1.0,...,1,1,2,1,1,2,1,2,1,1


In [7]:
cross_tab_prop

,id_aluno,nome_aluno,id_turma,escola,uf,municipio,cod_escola,nome_turma,E_ESC1V,E_ESC2V,...,E_PROF1C,E_PROF2C,E_FAM1C,E_FAM2C,E_COM1C,E_COM2C,E_COM3C,E_EST1C,E_EST2C,E_EST3C
0,14799,ADRYAN MAGNO DA SILVA NEVES,695,EEEFM DR OSWALDO PIANNA,RO,Porto Velho,11000937,None,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
1,18949,WEVERTON DE SOUZA DA SILVA,1059,E M E I F SANTA TEREZINHA,PA,Cametá,15547574,None,3.333333,3.0,...,3,2,3,2,1,1,3,2,2,1
2,19175,Monique Gomes de Souza,883,E M E F PROFESSORA DALILA LEAO,PA,Cametá,15070387,None,1.000000,3.0,...,1,1,1,1,1,1,1,2,1,1
3,19174,Elielson Rodrigues Viana,892,E M E F PROFESSORA DALILA LEAO,PA,Cametá,15070387,None,1.000000,4.0,...,1,1,1,2,1,1,3,2,1,3
4,19152,Jeferson Oliveira dos Santos,1064,E M E F PROFESSORA DALILA LEAO,PA,Cametá,15070387,None,3.000000,1.0,...,1,1,1,2,1,1,2,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,18920,ARLAN CARDOSO RODRIGUES,1059,E M E I F SANTA TEREZINHA,PA,Cametá,15547574,None,2.000000,1.0,...,1,1,3,1,1,3,1,2,1,1
58,18918,ALINE DA CRUZ MONTEIRO,1059,E M E I F SANTA TEREZINHA,PA,Cametá,15547574,None,1.333333,4.0,...,2,3,2,2,1,3,3,3,3,3
59,18917,ALEXANDRE LEÃO CANTÃO,1059,E M E I F SANTA TEREZINHA,PA,Cametá,15547574,None,5.500000,4.5,...,2,2,2,2,1,3,2,1,3,3
60,18916,ALESSANDRO PORTILHO DE MORAES,1059,E M E I F SANTA TEREZINHA,PA,Cametá,15547574,None,1.666667,1.0,...,1,1,2,1,1,2,1,2,1,1


In [8]:
filter_column = cross_tab_prop.columns.str.startswith("E_") | cross_tab_prop.columns.str.startswith("id") | cross_tab_prop.columns.str.startswith("cod")|cross_tab_prop.columns.str.startswith("nome")|cross_tab_prop.columns.str.startswith("escola") |cross_tab_prop.columns.str.startswith("uf")|cross_tab_prop.columns.str.startswith("municipio")
df_col = cross_tab_prop.loc[:,filter_column]
filter_column = df_col.columns.str.endswith("V") | df_col.columns.str.startswith("id") | df_col.columns.str.startswith("cod")|df_col.columns.str.startswith("nome")|df_col.columns.str.startswith("escola") |df_col.columns.str.startswith("uf")|df_col.columns.str.startswith("municipio")
df_col = df_col.loc[:, filter_column]
df_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id_aluno    62 non-null     int64  
 1   nome_aluno  62 non-null     object 
 2   id_turma    62 non-null     int64  
 3   escola      62 non-null     object 
 4   uf          62 non-null     object 
 5   municipio   62 non-null     object 
 6   cod_escola  62 non-null     int64  
 7   nome_turma  0 non-null      object 
 8   E_ESC1V     62 non-null     float64
 9   E_ESC2V     62 non-null     float64
 10  E_PROF1V    62 non-null     float64
 11  E_PROF2V    62 non-null     float64
 12  E_FAM1V     62 non-null     float64
 13  E_FAM2V     62 non-null     float64
 14  E_COM1V     62 non-null     float64
 15  E_COM2V     62 non-null     float64
 16  E_COM3V     62 non-null     float64
 17  E_EST1V     62 non-null     float64
 18  E_EST2V     62 non-null     float64
 19  E_EST3V     62 non-null     flo

In [9]:
df_col['E_ESCV'] = (df_col['E_ESC2V'] +df_col['E_ESC2V'])/2
df_col['E_PROFV'] = (df_col['E_PROF1V'] +df_col['E_PROF1V'])/2
df_col['E_FAMV'] = (df_col['E_FAM1V'] +df_col['E_FAM1V'])/2
df_col['E_COMV'] = (df_col['E_COM1V'] +df_col['E_COM2V']+df_col['E_COM3V'])/3
df_col['E_ESTV'] = (df_col['E_EST1V'] +df_col['E_EST2V']+df_col['E_EST3V'])/3


In [10]:
df_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id_aluno    62 non-null     int64  
 1   nome_aluno  62 non-null     object 
 2   id_turma    62 non-null     int64  
 3   escola      62 non-null     object 
 4   uf          62 non-null     object 
 5   municipio   62 non-null     object 
 6   cod_escola  62 non-null     int64  
 7   nome_turma  0 non-null      object 
 8   E_ESC1V     62 non-null     float64
 9   E_ESC2V     62 non-null     float64
 10  E_PROF1V    62 non-null     float64
 11  E_PROF2V    62 non-null     float64
 12  E_FAM1V     62 non-null     float64
 13  E_FAM2V     62 non-null     float64
 14  E_COM1V     62 non-null     float64
 15  E_COM2V     62 non-null     float64
 16  E_COM3V     62 non-null     float64
 17  E_EST1V     62 non-null     float64
 18  E_EST2V     62 non-null     float64
 19  E_EST3V     62 non-null     flo

In [11]:
filter = cross_tab_prop['escola']=='E M E F PROFESSORA DALILA LEAO'
escola_fator = cross_tab_prop[filter]
E_ESC1V = np.mean(escola_fator['E_ESC1V'])
E_ESC2V = np.mean(escola_fator['E_ESC2V'])
E_PROF1V = np.mean(escola_fator['E_PROF1V'])
E_PROF2V = np.mean(escola_fator['E_PROF2V'])
E_FAM1V = np.mean(escola_fator['E_FAM1V'])
E_FAM2V = np.mean(escola_fator['E_FAM2V'])
E_COM1V = np.mean(escola_fator['E_COM1V'])
E_COM2V = np.mean(escola_fator['E_COM2V'])
E_COM3V = np.mean(escola_fator['E_COM3V'])
E_EST1V = np.mean(escola_fator['E_EST1V'])
E_EST2V = np.mean(escola_fator['E_EST2V'])
E_EST3V = np.mean(escola_fator['E_EST3V'])

In [12]:
filter = (cross_tab_prop['escola']=='E M E F PROFESSORA DALILA LEAO') & (cross_tab_prop['id_turma']==1064)
turma_fator = cross_tab_prop[filter]

E_ESC1V = np.mean(turma_fator['E_ESC1V'])
E_ESC2V = np.mean(turma_fator['E_ESC2V'])
E_PROF1V = np.mean(turma_fator['E_PROF1V'])
E_PROF2V = np.mean(turma_fator['E_PROF2V'])
E_FAM1V = np.mean(turma_fator['E_FAM1V'])
E_FAM2V = np.mean(turma_fator['E_FAM2V'])
E_COM1V = np.mean(turma_fator['E_COM1V'])
E_COM2V = np.mean(turma_fator['E_COM2V'])
E_COM3V = np.mean(turma_fator['E_COM3V'])
E_EST1V = np.mean(turma_fator['E_EST1V'])
E_EST2V = np.mean(turma_fator['E_EST2V'])
E_EST3V = np.mean(turma_fator['E_EST3V'])

In [44]:
filter = (cross_tab_prop['escola']=='E M E F PROFESSORA DALILA LEAO') & (cross_tab_prop['id_turma']==1064) & (cross_tab_prop['nome_aluno']=='Jeferson Oliveira dos Santos')
aluno_fator = cross_tab_prop[filter]

E_ESC1V = np.mean(aluno_fator['E_ESC1V'])
E_ESC2V = np.mean(aluno_fator['E_ESC2V'])
E_PROF1V = np.mean(aluno_fator['E_PROF1V'])
E_PROF2V = np.mean(aluno_fator['E_PROF2V'])
E_FAM1V = np.mean(aluno_fator['E_FAM1V'])
E_FAM2V = np.mean(aluno_fator['E_FAM2V'])
E_COM1V = np.mean(aluno_fator['E_COM1V'])
E_COM2V = np.mean(aluno_fator['E_COM2V'])
E_COM3V = np.mean(aluno_fator['E_COM3V'])
E_EST1V = np.mean(aluno_fator['E_EST1V'])
E_EST2V = np.mean(aluno_fator['E_EST2V'])
E_EST3V = np.mean(aluno_fator['E_EST3V'])

In [ ]:
app = Dash(__name__)
#app = Flask(__name__)

DIV_STYLE = {
  "height": "900px",
  "display": "flex",
  "justify-content": "center",
  "align-items": "center",
}

CHART_BOX_STYLE = {
  "display": "flex",
  "justify-content": "center",
  "flex-wrap": "wrap",
  "align-items": "center",
}

CHART_STYLE = {
  "flex": "0 0 33.33%",
  "maxWidth": "33.33%",
}

app.layout = html.Div(
  [
    html.Div(
      [
        html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,3.65],"yellow":[3.65,5.33],"red":[5.33,7]}},
            value=E_ESC1V,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E-ESC1',
            max=7,
            min=0,
          ),
        ],
        style=CHART_STYLE),
        
        html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2],"yellow":[2,3.33],"red":[3.33,7]}},
            value=E_ESC2V,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E-ESC2',
            max=7,
            min=0,
          ),
        ],
        style=CHART_STYLE),
        
        html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2.33],"yellow":[2.33,4.33],"red":[4.33,7]}},
            value=E_PROF1V,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_PROF1',
            max=7,
            min=0,
          ),   
        ],
        style=CHART_STYLE),
        
        html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2.33],"yellow":[2.33,4],"red":[4,7]}},
            value=E_PROF2V,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_PROF2',
            max=7,
            min=0,
          ),
          
        ],
        style=CHART_STYLE),
        
        html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,3],"yellow":[3,5],"red":[5,7]}},
            value=E_FAM1V,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_FAM1',
            max=7,
            min=0,
          ),
            
      ],
      style=CHART_STYLE),
      
      html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,3.33],"yellow":[3.33,5],"red":[5,7]}},
            value=E_FAM2V,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_FAM2',
            max=7,
            min=0,
          ),
            
      ],
      style=CHART_STYLE),
          
      html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2],"yellow":[2,3.66],"red":[3.66,7]}},
            value=E_COM1V,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_COM1',
            max=7,
            min=0,
          ),
            
      ],
      style=CHART_STYLE),
    
     html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2],"yellow":[2,3.66],"red":[3.66,7]}},
            value=E_COM2V,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_COM2',
            max=7,
            min=0,
          ),
            
      ],
      style=CHART_STYLE),
          
      html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2],"yellow":[2,3.66],"red":[3.66,7]}},
            value=E_COM3V,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_COM3',
            max=7,
            min=0,
          ),
            
      ],
      style=CHART_STYLE),
          
    html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2],"yellow":[2,4],"red":[4,7]}},
            value=E_EST1V,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_EST1',
            max=7,
            min=0,
          ),
            
      ],
      style=CHART_STYLE),
    
    html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2.33],"yellow":[2.33,4],"red":[4,7]}},
            value=E_EST2V,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_EST2',
            max=7,
            min=0,
          ),
            
      ],
      style=CHART_STYLE),
          
       html.Div([
          daq.Gauge(
            color={"gradient":True,"ranges":{"green":[0,2],"yellow":[2,3.33],"red":[3.33,7]}},
            value=E_EST3V,
            scale={'start': 0, 'interval': 1, 'labelInterval': 1},
            label='E_EST3',
            max=7,
            min=0,
          ),
            
      ],
      style=CHART_STYLE),
          
      ],
      style=CHART_BOX_STYLE,
    )
  ],
  style=DIV_STYLE,  
)

if __name__ == '__main__':
    app.run()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [06/Dec/2022 22:15:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 22:15:55] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 22:15:55] "GET /_dash-dependencies HTTP/1.1" 200 -
